In [ ]:
%rm -rf LLaMA-Factory
!git clone https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install .

In [ ]:
from llmtuner import run_exp, ChatModel, export_model


import torch
torch.cuda.is_available()

We are using Mistral-7b-Instruct V2 AI, Feel free to explore other Mistral models at: https://huggingface.co/mistralai Also you can explore other supported architectures in LLaMA Factory official repo

Here we are using identity,alpaca_gpt4_en,alpaca_gpt4_zh datasets to fine-tune the model, you can explore other datasets in hugging face according to the need.

In [ ]:
# Here we train the model and export it to a directory called "E2E_Mistral-7B-ChatBot"
run_exp(dict(
  stage="sft",
  do_train=True,
  model_name_or_path="mistralai/Mistral-7B-Instruct-v0.2",
  dataset="identity,alpaca_gpt4_en,alpaca_gpt4_zh",
  template="mistral",
  finetuning_type="lora",
  lora_target="all",
  output_dir="E2E_Mistral-7B-ChatBot",
  per_device_train_batch_size=4,
  gradient_accumulation_steps=4,
  lr_scheduler_type="cosine",
  logging_steps=10,
  save_steps=100,
  learning_rate=1e-4,
  num_train_epochs=5.0,
  max_samples=500,
  max_grad_norm=1.0,
  fp16=True,
))

Note that we have a folder created inside the LLaMA-Factory with the trained model checkpoints. We can export the fine tuned model to "E2E-ChatBot" directory using the below code

In [ ]:
export_model(dict(
  model_name_or_path="mistralai/Mistral-7B-Instruct-v0.2",
  adapter_name_or_path="E2E_Mistral-7B-ChatBot",
  finetuning_type="lora",
  template="mistral",
  export_dir="E2E-ChatBot",
))

Now Let's play with the trained model

In [ ]:
E2E_Chat_Bot = ChatModel(
    dict(
  model_name_or_path="mistralai/Mistral-7B-Instruct-v0.2",
  adapter_name_or_path="E2E_Mistral-7B-ChatBot",
  finetuning_type="lora",
  template="mistral",
))

In [ ]:
messages = []
while True:
  query = input("\nUser: ")
  if query.strip() == "exit":
    break
  if query.strip() == "clear":
    messages = []
    continue

  messages.append({"role": "user", "content": query})
  print("E2E_Chat_Bot: ", end="", flush=True)
  response = ""
  for new_text in E2E_Chat_Bot.stream_chat(messages):
    response += new_text
  for r in response.split('.'):
    print(r.strip(), flush=True)
  print()
  messages.append({"role": "assistant", "content": response})